# Sample Analysis

4 different seeds with the following:

* Seed A: 0
* Seed B: 42
* Seed C: 123
* Seed D: 2025

652 samples with the following:

* TP: 163
* TN: 163
* FP: 163
* FN: 163

### Samples df

In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dtale
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy.stats import spearmanr, kendalltau
from sklearn.metrics import jaccard_score


from master_thesis.config import PROCESSED_DATA_DIR, load_dataframe_from_pickle, save_dataframe_as_pickle

In [2]:
working_df = load_dataframe_from_pickle(
    PROCESSED_DATA_DIR / "samples_with_XAI_applied.pkl"
)

working_df.head(5)

,text,error,cardiff_label,cardiff_score,lime_cardiff,shap_cardiff,seed
0,"India is one nasty, dirty, filthy, germsy, law...",NaN,HATE,0.994301,"country: -0.13682444945684807, India: -0.11131...",": -1.309963408857584e-07, India : -0.137895100...",0
1,solo que seras bombardeas y destruidos como lo...,NaN,HATE,0.957913,"virus: -0.6440492827882649, a: 0.2838806631586...",": 3.37468518409878e-05, s: -0.0331062855402706...",0
2,@user @user You cannot argue with a retard jer...,NaN,HATE,0.994864,"retard: -0.6439715209019052, beggar: -0.128180...",": 1.7441052477806807e-06, @: 0.014520991622703...",0
3,Can I just pound that pussy an pull your hair....,NaN,HATE,0.977588,"your: -0.5850116056602925, tits: -0.1649002364...",": 9.252107702195644e-08, Can : 0.0115244268963...",0
4,you do not live in america nazi faggot but you...,NaN,HATE,0.985650,"faggot: -0.8957589632472828, nazi: -0.05553477...",": 1.1423340765759349e-05, you : 0.001079929454...",0


In [3]:
working_df['error'].value_counts()

Series([], Name: count, dtype: int64)

In [4]:
working_df['cardiff_hatespeech'] = working_df['cardiff_label'].apply(lambda x: 1 if x == 'HATE' else 0)

In [5]:
working_df.drop(columns=['cardiff_label'], inplace=True)

### Full df

In [6]:
full_hatespeech_df = load_dataframe_from_pickle(
    PROCESSED_DATA_DIR / "annotated_and_targeted_hatespeech.pkl"
)

full_hatespeech_df.head(5)

,id_new,text,extracted_target,label_hatespeech_binary_offensive_not_included,label_hatespeech_binary_offensive_included,label_normal_offensive_hatespeech,platform,original_dataset_title,original_id,original_label,original_target
0,1,i dont think im getting my baby them white 9 h...,{none},not_hatespeech,normal,normal,twitter,HateXplain,1179055004553900032,normal,[None]
1,2,we cannot continue calling ourselves feminists...,{none},not_hatespeech,normal,normal,twitter,HateXplain,1179063826874032128,normal,[None]
2,3,nawt yall niggers ignoring me,{race},not_hatespeech,normal,normal,twitter,HateXplain,1178793830532956161,normal,[African]
3,4,<user> i am bit confused coz chinese ppl can n...,{race},hatespeech,hatespeech/offensive,hatespeech,twitter,HateXplain,1179088797964763136,hatespeech,[Asian]
4,5,this bitch in whataburger eating a burger with...,"{race, gender}",hatespeech,hatespeech/offensive,hatespeech,twitter,HateXplain,1179085312976445440,hatespeech,"[Caucasian, Women]"


In [7]:
columns_to_keep = ['id_new', 'text', 'extracted_target','label_hatespeech_binary_offensive_included','platform']
adjusted_hatespeech_df = full_hatespeech_df[columns_to_keep]

In [8]:
adjusted_hatespeech_df['is_hatespeech'] = adjusted_hatespeech_df['label_hatespeech_binary_offensive_included'].apply(lambda x: 1 if x == 'hatespeech/offensive' else 0)
adjusted_hatespeech_df['id'] = adjusted_hatespeech_df['id_new']

/tmp/ipykernel_186238/2421778535.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_186238/2421778535.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
adjusted_hatespeech_df.drop(columns=['label_hatespeech_binary_offensive_included','id_new'], inplace=True)

/tmp/ipykernel_186238/3016290075.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
full_df = pd.merge(working_df, adjusted_hatespeech_df, on='text', how='inner')

In [11]:


d = dtale.show(full_df)
d.open_browser()

In [15]:
full_df = full_df.drop('error', axis=1)

In [20]:
full_df.columns

Index(['text', 'cardiff_score', 'lime_cardiff', 'shap_cardiff', 'seed',
       'cardiff_hatespeech', 'extracted_target', 'platform', 'is_hatespeech',
       'id'],
      dtype='object')

In [25]:
full_df.head()

,text,cardiff_score,lime_cardiff,shap_cardiff,seed,cardiff_hatespeech,extracted_target,platform,is_hatespeech,id
0,"India is one nasty, dirty, filthy, germsy, law...",0.994301,"country: -0.13682444945684807, India: -0.11131...",": -1.309963408857584e-07, India : -0.137895100...",0,1,{nationality},youtube,1,53513
1,solo que seras bombardeas y destruidos como lo...,0.957913,"virus: -0.6440492827882649, a: 0.2838806631586...",": 3.37468518409878e-05, s: -0.0331062855402706...",0,1,{religion},gab,1,8335
2,@user @user You cannot argue with a retard jer...,0.994864,"retard: -0.6439715209019052, beggar: -0.128180...",": 1.7441052477806807e-06, @: 0.014520991622703...",0,1,"{nationality, disability}",twitter,1,64112
3,Can I just pound that pussy an pull your hair....,0.977588,"your: -0.5850116056602925, tits: -0.1649002364...",": 9.252107702195644e-08, Can : 0.0115244268963...",0,1,{gender},reddit,1,22034
4,you do not live in america nazi faggot but you...,0.985650,"faggot: -0.8957589632472828, nazi: -0.05553477...",": 1.1423340765759349e-05, you : 0.001079929454...",0,1,{sexuality},gab,1,18143


In [27]:
full_df[['lime_cardiff','shap_cardiff']]

,lime_cardiff,shap_cardiff
0,"country: -0.13682444945684807, India: -0.11131...",": -1.309963408857584e-07, India : -0.137895100..."
1,"virus: -0.6440492827882649, a: 0.2838806631586...",": 3.37468518409878e-05, s: -0.0331062855402706..."
2,"retard: -0.6439715209019052, beggar: -0.128180...",": 1.7441052477806807e-06, @: 0.014520991622703..."
3,"your: -0.5850116056602925, tits: -0.1649002364...",": 9.252107702195644e-08, Can : 0.0115244268963..."
4,"faggot: -0.8957589632472828, nazi: -0.05553477...",": 1.1423340765759349e-05, you : 0.001079929454..."
...,...,...
2603,"kike: -0.14024712127870378, jesus: 0.124653868...",": 4.0978193283081055e-08, um : 0.0233726911246..."
2604,"pathetic: -0.24076589739412038, a: 0.221748126...",": 4.0357311566670734e-07, Mom: 0.0108912767221..."
2605,"nigger: -0.537177335386766, say: 0.44650229997...",": -2.2202730178833008e-06, Black : 0.008567720..."
2606,"retard: -0.6656245064558355, I: 0.324688800703...",": -1.862645149230957e-08, pretty : 0.104445570..."


In [28]:


d = dtale.show(full_df[['lime_cardiff','shap_cardiff']])
d.open_browser()

In [12]:
type(full_df['lime_cardiff'][0])

str

### Ranking

In [29]:
import pandas as pd

def parse_and_rank_features(feature_string):
    """
    Parses a string of feature contributions, ranks them by absolute weight,
    and returns the top 5.

    Args:
        feature_string (str): A string in the format like "feat1: val1, feat2: val2, ..."
                              or ": val1, feat2: val2, ..." or "val1, feat2: val2, ...".
                              It can also be None, NaN, or an empty string.

    Returns:
        list: A list of up to 5 tuples, where each tuple is (feature_name, original_weight).
              The list is sorted by the absolute value of the weights in descending order.
              Returns an empty list if the input is invalid or no features are parsed.
    """
    if not isinstance(feature_string, str) or not feature_string.strip():
        # Handles None, NaN, empty string, or string with only whitespace
        return []

    # Split the main string by commas to get individual feature parts
    raw_items = feature_string.split(',')
    
    feature_contributions = []
    for raw_item in raw_items:
        item = raw_item.strip()  # Remove leading/trailing whitespace from the part
        if not item:
            continue  # Skip if the part is empty after stripping

        # We use rpartition(':') to split the item by the *last* colon.
        # This is robust in case a feature name itself contains a colon.
        # - If "name:value", parts are ('name', ':', 'value')
        # - If ":value", parts are ('', ':', 'value')
        # - If "value" (no colon), parts are ('value', '', '')
        name_part, separator, value_part = item.rpartition(':')
        
        feature_name = ""
        weight_str = ""

        if separator == ':':
            # This is the common case, e.g., "feature_name: 0.5" or ": 0.5"
            feature_name = name_part.strip()
            weight_str = value_part.strip()
        else:
            # No colon was found in this item. Assume the entire item is a weight
            # for an unnamed feature. e.g. "0.5" becomes feature="", weight="0.5"
            feature_name = ""  # Default to an empty string for the feature name
            weight_str = name_part.strip() # The whole item (name_part here) is the weight string

        if not weight_str:
            # This occurs if the item was like "feature_name:" (empty after colon)
            # or if after parsing, weight_str ended up empty.
            # print(f"Warning: No weight value found for item '{item}'. Skipping.") # Optional: for debugging
            continue
            
        try:
            weight = float(weight_str)
            feature_contributions.append({
                'feature': feature_name, 
                'weight': weight, 
                'abs_weight': abs(weight)
            })
        except ValueError:
            # This occurs if the weight_str cannot be converted to a float.
            # print(f"Warning: Could not parse weight from '{weight_str}' in item '{item}'. Skipping.") # Optional: for debugging
            continue
            
    # Sort the collected features by their absolute weight in descending order
    sorted_features = sorted(feature_contributions, key=lambda x: x['abs_weight'], reverse=True)
    
    # Get the top 5 (or fewer if not enough features were parsed)
    top_5 = [(f['feature'], f['weight']) for f in sorted_features[:5]]
    
    return top_5

In [30]:
# Apply the function to the 'lime_cardiff' column
full_df['lime_top5'] = full_df['lime_cardiff'].apply(parse_and_rank_features)

# Apply the function to the 'shap_cardiff' column
full_df['shap_top5'] = full_df['shap_cardiff'].apply(parse_and_rank_features)

In [32]:

d = dtale.show(full_df[['lime_top5','shap_top5']])
d.open_browser()

In [33]:
d = dtale.show(full_df)
d.open_browser()

In [34]:
d.kill()

2025-05-07 15:53:39,262 - INFO     - Executing shutdown...
2025-05-07 15:53:39,263 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer


In [41]:
type(full_df['lime_top5'][0][0][0])

str

In [42]:
def extract_feature_names(top_features_list):
    """
    Extracts only the feature names from a list of (feature_name, weight) tuples.
    The order of feature names is preserved.

    Args:
        top_features_list (list): A list of tuples, where each tuple is expected
                                  to be (feature_name, weight). This is typically
                                  the output from the parse_and_rank_features function.

    Returns:
        list: A list of strings (feature names). Returns an empty list if the
              input is not a list or is empty.
    """
    if not isinstance(top_features_list, list):
        # Handles cases where the input might not be a list (e.g., if an error occurred upstream)
        return []
    
    # Extracts the first element (feature name) from each tuple in the list
    # Ensures that it only processes actual tuples with at least one element
    feature_names = [
        feature_tuple[0] 
        for feature_tuple in top_features_list 
        if isinstance(feature_tuple, tuple) and len(feature_tuple) > 0
    ]
    return feature_names

In [43]:
# Apply the new function to the 'lime_top5' column
full_df['lime_top5_names'] = full_df['lime_top5'].apply(extract_feature_names)

# Apply the new function to the 'shap_top5' column
full_df['shap_top5_names'] = full_df['shap_top5'].apply(extract_feature_names)

In [48]:
d = dtale.show(full_df[['id','lime_top5_names','shap_top5_names','seed']])
d.open_browser()

In [49]:
d.kill()

2025-05-07 16:50:04,894 - INFO     - Executing shutdown...
2025-05-07 16:50:04,894 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer


### Applying Metrics

In [51]:
def calculate_jaccard_similarity(list1, list2):
    """
    Calculates the Jaccard similarity between two lists of feature names.
    """
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if union == 0:
        return 1.0 # Or 0.0, depending on how you want to treat two empty lists
    return intersection / union

In [52]:

def calculate_rank_correlation(list1, list2, method='spearman'):
    """
    Calculates Spearman's rank correlation or Kendall's Tau for two lists of feature names.
    Features are ranked by their position in the list.
    Features not present in one list but present in the other are handled by
    assigning them a rank lower than any item in the list.
    
    Args:
        list1 (list): First list of feature names (ordered).
        list2 (list): Second list of feature names (ordered).
        method (str): 'spearman' or 'kendall'.

    Returns:
        float: The correlation coefficient. Returns NaN if lists are too short or have no overlap.
    """
    all_features = list(set(list1) | set(list2))
    if not all_features:
        return 1.0 # Or NaN, if both lists are empty

    # Create ranks for each list.
    # Features present get their 0-indexed position as rank.
    # Features not present get a rank equal to the length of the list (lower importance).
    ranks1 = {feature: i for i, feature in enumerate(list1)}
    ranks2 = {feature: i for i, feature in enumerate(list2)}

    vec1 = [ranks1.get(f, len(list1)) for f in all_features]
    vec2 = [ranks2.get(f, len(list2)) for f in all_features]

    if len(vec1) < 2 or len(vec2) < 2 : # Correlation is not well-defined for single elements
        return None # Or handle as per your needs, e.g. 0 or 1 if lists are identical but short

    # Handle cases where all ranks are the same (no variance), scipy might return NaN or error
    if len(set(vec1)) <= 1 or len(set(vec2)) <= 1:
         if vec1 == vec2: # If they are identical constant lists
             return 1.0
         else: # Different constant lists or one constant, one not
             return 0.0 # Or None, as correlation is tricky here

    if method == 'spearman':
        corr, _ = spearmanr(vec1, vec2)
    elif method == 'kendall':
        corr, _ = kendalltau(vec1, vec2)
    else:
        raise ValueError("Method must be 'spearman' or 'kendall'")
    
    return corr

In [62]:
results = [] # To store the comparison results

# Group by 'id'
grouped_by_id = full_df.groupby('id')

for id_val, group in grouped_by_id:
    if len(group) < 2: # Need at least two different seeds for comparison
        continue

    # Convert rows to a list of tuples (seed, feature_list) for easier iteration
    # Focusing on LIME for this example:
    seed_feature_lists_lime = []
    for _, row in group.iterrows():
        seed_feature_lists_lime.append((row['seed'], row['lime_top5_names']))
    
    seed_feature_lists_shap = []
    for _, row in group.iterrows():
        seed_feature_lists_shap.append((row['seed'], row['shap_top5_names']))


    # Pairwise comparison for LIME lists within the current 'id' group
    for i in range(len(seed_feature_lists_lime)):
        for j in range(i + 1, len(seed_feature_lists_lime)):
            seed1, list1 = seed_feature_lists_lime[i]
            seed2, list2 = seed_feature_lists_lime[j]

            # Ensure lists are actually lists (they should be from .apply(extract_feature_names))
            if not isinstance(list1, list) or not isinstance(list2, list):
                print(f"Warning: Skipping comparison for id {id_val}, seeds {seed1}-{seed2} due to non-list feature set.")
                continue
            
            jaccard_sim = calculate_jaccard_similarity(list1, list2)
            spearman_corr = calculate_rank_correlation(list1, list2, method='spearman')
            kendall_tau_val = calculate_rank_correlation(list1, list2, method='kendall')

            results.append({
                'id': id_val,
                'seed1': seed1,
                'seed2': seed2,
                'comparison_type': 'lime_top5_names', # Specify which list type was compared
                'jaccard_similarity': jaccard_sim,
                'spearman_correlation': spearman_corr,
                'kendall_tau': kendall_tau_val
            })
    
    for i in range(len(seed_feature_lists_shap)):
        for j in range(i + 1, len(seed_feature_lists_shap)):
            seed1, list1_shap = seed_feature_lists_shap[i]
            seed2, list2_shap = seed_feature_lists_shap[j]
            
            if not isinstance(list1_shap, list) or not isinstance(list2_shap, list):
                continue # Skip if data is not as expected

            jaccard_sim_shap = calculate_jaccard_similarity(list1_shap, list2_shap)
            spearman_corr_shap = calculate_rank_correlation(list1_shap, list2_shap, method='spearman')
            kendall_tau_shap = calculate_rank_correlation(list1_shap, list2_shap, method='kendall')

            results.append({
                'id': id_val,
                'seed1': seed1,
                'seed2': seed2,
                'comparison_type': 'shap_top5_names',
                'jaccard_similarity': jaccard_sim_shap,
                'spearman_correlation': spearman_corr_shap,
            'kendall_tau': kendall_tau_shap
            })

In [63]:
# Convert results to a DataFrame
results_df = pd.DataFrame(results)

print("\nComparison Results:")
print(results_df.to_string())


Comparison Results:
         id  seed1  seed2  comparison_type  jaccard_similarity  spearman_correlation  kendall_tau
0         7      0     42  lime_top5_names            0.666667              0.771429     0.600000
1         7      0    123  lime_top5_names            0.666667              0.828571     0.733333
2         7      0   2025  lime_top5_names            0.666667              0.828571     0.733333
3         7     42    123  lime_top5_names            0.666667              0.942857     0.866667
4         7     42   2025  lime_top5_names            0.666667              0.942857     0.866667
5         7    123   2025  lime_top5_names            1.000000              1.000000     1.000000
6         7      0     42  shap_top5_names            1.000000              1.000000     1.000000
7         7      0    123  shap_top5_names            1.000000              1.000000     1.000000
8         7      0   2025  shap_top5_names            1.000000              1.000000     1.000000

In [64]:
d = dtale.show(results_df)
d.open_browser()

2025-05-07 17:05:35,210 - ERROR    - Exception occurred while processing request: list index out of range
Traceback (most recent call last):
  File "/home/takosaga/miniconda3/envs/master_thesis/lib/python3.10/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/takosaga/miniconda3/envs/master_thesis/lib/python3.10/site-packages/dtale/views.py", line 2998, in get_data
    {IDX_COL: sub_range[0]}, sub_df[0]
IndexError: list index out of range
